In [1]:
from nltk.corpus import wordnet as wn
all_nouns = list(wn.all_synsets('n'))

In [2]:
import numpy as np
import cPickle as pickle
word_embedding_matrix = np.load('/u/vendrov/qanda/glove/glove_matrix.npy')
word2index = pickle.load(open('/u/vendrov/qanda/glove/glove_word_index.pkl', 'r'))

In [3]:
import numpy as np
import re
from collections import OrderedDict
all_words = []
all_w2i = OrderedDict()
entity_lengths = [] # store length, in words, for each entity
entity2word = [] # store indices of words in all entities, in order



for i in range(len(all_nouns)):
    synset = all_nouns[i]
    # the embedding of the synset is the embedding of the first lemma
    words = []
    for lemma in synset.lemma_names():
        words = filter(lambda w: len(w) > 0 and w != 's', re.split('[\W|_]+', lemma))
        if words: break;
    assert(words)
    entity_lengths.append(len(words))
    for word in words:
        if word not in all_w2i:
            all_w2i[word] = len(all_words)
            all_words.append(word)
        entity2word.append(all_w2i[word])
        
        
in_w2v = np.zeros((len(all_words),))
word_embeddings = np.zeros((len(all_words), word_embedding_matrix.shape[1]))
for i in range(len(all_words)):
    word = all_words[i]
    if word in word2index:
        word_embeddings[i] = word_embedding_matrix[word2index[word]]
        in_w2v[i] = 1

In [4]:
### get hypernyms

# get mapping of synset id to index
id2index = {}
for i in range(len(all_nouns)):
    id2index[all_nouns[i].name()] = i
    
# get hypernym relations
hypernyms = []
for synset in all_nouns:
    for h in synset.hypernyms() + synset.instance_hypernyms():
        hypernyms.append([id2index[synset.name()], id2index[h.name()]])
hypernyms = np.array(hypernyms)

In [8]:
all_nouns[0].substance_holonyms()

[]

In [9]:
### get holonyms
holonyms = []
for synset in all_nouns:
    for h in synset.substance_holonyms() + synset.member_holonyms() + synset.part_holonyms():
        holonyms.append([id2index[synset.name()], id2index[h.name()]])
holonyms = np.array(holonyms)

In [18]:
c

22196

In [10]:
slices = []
offset = 0
for length in entity_lengths:
    slices.append((offset, offset+length))
    offset += length

[1, 2, 1, 1, 1, 1, 1, 2, 1, 1]


In [13]:
import h5py
f = h5py.File('wordnet.h5', 'w')

In [14]:
f.create_dataset('embeddings', data=word_embeddings)
f.create_dataset('in_w2v', data=in_w2v)
f.create_dataset('hypernyms', data = hypernyms)
f.create_dataset('holonyms', data = holonyms)
f.create_dataset('slices', data = np.array(slices))
f.create_dataset('entity2word', data = np.array(entity2word))

<HDF5 dataset "entity2word": shape (116007,), type "<i8">

In [15]:
f.close()

In [16]:
# save list of synset names
names = map(lambda s: s.name(), all_nouns)
import json
json.dump(names, open('synset_names.json', 'w'))

In [17]:
np.linalg.norm(word_embeddings[1])

6.4585522095627734